<a href="https://colab.research.google.com/github/hareshbaskaran/AmazonDB-FastApi/blob/master/sarcasm_detection_reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sarcasm Detection Using Neural Networks

Dataset link : https://drive.google.com/file/d/15u7gMIgbPhIdFuuljiIY4ub4lLdwpxcc/view

##Importing Dataset and required libraries

In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_df= pd.read_csv("/content/drive/MyDrive/train-balanced-sarcasm.csv",)
train_df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [ ]:
len(train_df),train_df.index.shape[-1]


(1010826, 1010826)

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing,metrics,manifold
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,cross_val_predict
from imblearn.over_sampling import ADASYN,SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
import collections
import matplotlib.patches as mpatches
from sklearn.metrics import accuracy_score
%matplotlib inline
from sklearn.preprocessing import RobustScaler
import xgboost
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
from collections import Counter
from sklearn.model_selection import StratifiedKFold,KFold,StratifiedShuffleSplit
from nltk import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD,SparsePCA
from sklearn.metrics import classification_report,confusion_matrix
from nltk.tokenize import word_tokenize
from collections import defaultdict
from collections import Counter
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
import nltk
from nltk.corpus import stopwords
import string
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import keras
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from keras.layers import Dense, BatchNormalization
from tensorflow.keras.preprocessing.sequence import pad_sequences


## Preprocessing

In [ ]:
print("Sntiment - ".format(),train_df.shape)

In [ ]:
train_df = train_df.dropna()
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1010773 entries, 0 to 1010825
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   label           1010773 non-null  int64 
 1   comment         1010773 non-null  object
 2   author          1010773 non-null  object
 3   subreddit       1010773 non-null  object
 4   score           1010773 non-null  int64 
 5   ups             1010773 non-null  int64 
 6   downs           1010773 non-null  int64 
 7   date            1010773 non-null  object
 8   created_utc     1010773 non-null  object
 9   parent_comment  1010773 non-null  object
dtypes: int64(4), object(6)
memory usage: 84.8+ MB


In [ ]:
dict_df={
    "label":train_df["label"],
    "comment":train_df["comment"],
}
df = pd.DataFrame(dict_df)
df["comment"] = df["comment"].astype(str)
df.head()

,label,comment
0,0,NC and NH.
1,0,You do know west teams play against west teams...
2,0,"They were underdogs earlier today, but since G..."
3,0,"This meme isn't funny none of the ""new york ni..."
4,0,I could use one of those tools.


In [ ]:
#lowercasing 
df["text_lower"] = df["comment"].str.lower()
df.head(10)

,label,comment,text_lower
0,0,NC and NH.,nc and nh.
1,0,You do know west teams play against west teams...,you do know west teams play against west teams...
2,0,"They were underdogs earlier today, but since G...","they were underdogs earlier today, but since g..."
3,0,"This meme isn't funny none of the ""new york ni...","this meme isn't funny none of the ""new york ni..."
4,0,I could use one of those tools.,i could use one of those tools.
5,0,"I don't pay attention to her, but as long as s...","i don't pay attention to her, but as long as s..."
6,0,Trick or treating in general is just weird...,trick or treating in general is just weird...
7,0,Blade Mastery+Masamune or GTFO!,blade mastery+masamune or gtfo!
8,0,"You don't have to, you have a good build, buy ...","you don't have to, you have a good build, buy ..."
9,0,I would love to see him at lolla.,i would love to see him at lolla.


In [ ]:
#stopword removal
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [ ]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["text_wo_stop"] = df["text_lower"].apply(lambda text: remove_stopwords(text))
df.head()

,label,comment,text_lower,text_wo_stop
0,0,NC and NH.,nc and nh.,nc nh.
1,0,You do know west teams play against west teams...,you do know west teams play against west teams...,know west teams play west teams east teams right?
2,0,"They were underdogs earlier today, but since G...","they were underdogs earlier today, but since g...","underdogs earlier today, since gronk's announc..."
3,0,"This meme isn't funny none of the ""new york ni...","this meme isn't funny none of the ""new york ni...","meme funny none ""new york nigga"" ones are."
4,0,I could use one of those tools.,i could use one of those tools.,could use one tools.


In [ ]:
#lemmatization
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df["text_lemmatized"] = df["text_wo_stop"].apply(lambda text: lemmatize_words(text))
df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...


,label,comment,text_lower,text_wo_stop,text_lemmatized
0,0,NC and NH.,nc and nh.,nc nh.,nc nh.
1,0,You do know west teams play against west teams...,you do know west teams play against west teams...,know west teams play west teams east teams right?,know west team play west team east team right?
2,0,"They were underdogs earlier today, but since G...","they were underdogs earlier today, but since g...","underdogs earlier today, since gronk's announc...","underdog earlier today, since gronk's announce..."
3,0,"This meme isn't funny none of the ""new york ni...","this meme isn't funny none of the ""new york ni...","meme funny none ""new york nigga"" ones are.","meme funny none ""new york nigga"" one are."
4,0,I could use one of those tools.,i could use one of those tools.,could use one tools.,could use one tools.


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1010773 entries, 0 to 1010825
Data columns (total 5 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   label            1010773 non-null  int64 
 1   comment          1010773 non-null  object
 2   text_lower       1010773 non-null  object
 3   text_wo_stop     1010773 non-null  object
 4   text_lemmatized  1010773 non-null  object
dtypes: int64(1), object(4)
memory usage: 46.3+ MB


##Tokenization of Comments

In [ ]:
from nltk.lm.preprocessing import pad_sequence
tokenizer=Tokenizer(num_words=1000,oov_token='<OOV>')
tokenizer.fit_on_texts(df["text_lemmatized"])

In [ ]:
df["tokens"] = tokenizer.texts_to_sequences(df["text_lemmatized"])
df["length"] = df["tokens"].apply(lambda x: len(x))
df=df[(df['length']<=100) &(df['length']>10)]
df.info()
comments = pad_sequences(df["tokens"])
labels = df["label"]

print(len(comments[0]))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119526 entries, 2 to 1010821
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   label            119526 non-null  int64 
 1   comment          119526 non-null  object
 2   text_lower       119526 non-null  object
 3   text_wo_stop     119526 non-null  object
 4   text_lemmatized  119526 non-null  object
 5   tokens           119526 non-null  object
 6   length           119526 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 7.3+ MB
99


## Test train split


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(comments, labels, train_size = 0.8, random_state = 42, shuffle = True)


##CNN Model

In [ ]:
cnn_model = keras.Sequential([
    keras.layers.Embedding(50000, 64),
    keras.layers.Conv1D(128, 5, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.GlobalMaxPooling1D(),
    keras.layers.BatchNormalization(),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

cnn_model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(0.0002),metrics=['accuracy'])
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          3200000   
                                                                 
 conv1d (Conv1D)             (None, None, 128)         41088     
                                                                 
 dropout (Dropout)           (None, None, 128)         0         
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 flatten (Flatten)           (None, 128)               0

In [ ]:
cnn_model.fit(X_train,y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)


Epoch 1/10
2989/2989 [==============================] - 50s 13ms/step - loss: 0.6050 - accuracy: 0.6690 - val_loss: 0.7452 - val_accuracy: 0.6014
Epoch 2/10
2989/2989 [==============================] - 21s 7ms/step - loss: 0.5647 - accuracy: 0.7103 - val_loss: 0.8530 - val_accuracy: 0.5961
Epoch 3/10
2989/2989 [==============================] - 20s 7ms/step - loss: 0.5515 - accuracy: 0.7189 - val_loss: 0.9513 - val_accuracy: 0.5804
Epoch 4/10
2989/2989 [==============================] - 19s 6ms/step - loss: 0.5410 - accuracy: 0.7270 - val_loss: 0.9005 - val_accuracy: 0.5950
Epoch 5/10
2989/2989 [==============================] - 21s 7ms/step - loss: 0.5312 - accuracy: 0.7338 - val_loss: 0.8423 - val_accuracy: 0.6227
Epoch 6/10
2989/2989 [==============================] - 20s 7ms/step - loss: 0.5206 - accuracy: 0.7414 - val_loss: 0.8017 - val_accuracy: 0.6392
Epoch 7/10
2989/2989 [==============================] - 19s 6ms/step - loss: 0.5102 - accuracy: 0.7472 - val_loss: 0.8457 - val_a

## LSTM Model

In [ ]:
lstm_model = keras.Sequential([
    keras.layers.Embedding(50000, 64),
    keras.layers.Conv1D(128,5,activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dropout(0.2),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

lstm_model.compile(loss="binary_crossentropy", optimizer='rmsprop', metrics=["binary_accuracy"])
lstm_model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 64)          3200000   
                                                                 
 conv1d_1 (Conv1D)           (None, None, 128)         41088     
                                                                 
 dropout_1 (Dropout)         (None, None, 128)         0         
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 128)              512       
 hNormalization)                                      

In [ ]:
lstm_model.fit(X_train,y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)

Epoch 1/10
2989/2989 [==============================] - 51s 17ms/step - loss: 0.4877 - binary_accuracy: 0.7704 - val_loss: 0.5826 - val_binary_accuracy: 0.7049
Epoch 2/10
2989/2989 [==============================] - 38s 13ms/step - loss: 0.4790 - binary_accuracy: 0.7762 - val_loss: 0.6355 - val_binary_accuracy: 0.6754
Epoch 3/10
2989/2989 [==============================] - 48s 16ms/step - loss: 0.4717 - binary_accuracy: 0.7792 - val_loss: 0.6020 - val_binary_accuracy: 0.7092
Epoch 4/10
2989/2989 [==============================] - 40s 13ms/step - loss: 0.4638 - binary_accuracy: 0.7858 - val_loss: 0.6404 - val_binary_accuracy: 0.7010
Epoch 5/10
2989/2989 [==============================] - 43s 14ms/step - loss: 0.4557 - binary_accuracy: 0.7915 - val_loss: 0.5890 - val_binary_accuracy: 0.7012
Epoch 6/10
2989/2989 [==============================] - 41s 14ms/step - loss: 0.4457 - binary_accuracy: 0.7972 - val_loss: 0.6150 - val_binary_accuracy: 0.6920
Epoch 7/10
2989/2989 [==================

In [ ]:
text = ["I'm going to VIT, hope i get some knowledge"]
text[0] = remove_stopwords(text[0])
text[0] = lemmatize_words(text[0])
seq = tokenizer.texts_to_sequences(text)
seq = pad_sequences(seq)
pred = lstm_model.predict(seq)


if pred[0]<=0.10:
    print(" NO SARCASM ")
else:
    print(" SARCASM DETECTED ")

1/1 [==============================] - 0s 19ms/step
 SARCASM DETECTED 
